## Information Parsing Notebook

This notebook extracts the facebook page, emails, phone numbers, websites, etc. from the 'Group URL/Contact Info'

In [3]:
import numpy as np
import pandas as pd
import nltk
import string
import re
import requests
import urllib

In [46]:
path = '/Users/michaelbinkley/Downloads/Local Action Groups - All Jan 14.csv'
data = pd.read_csv(path)

In [53]:
group_info = map(str, data['Group URL/Contact Info'].tolist())

In [55]:
def facebook_url(s):
    s = s.lower().split()
    r1 = r'https://www.facebook.com/groups/.*/[^\s]'
    r2 = r'https://www.facebook.com/.*[^\s]'
    r3 = r'www.facebook.com/.*[^\s]'
    r4 = r'https://facebook.com/.*[^\s]'
    r5 = r'.*facebook.com/.*[^\s]'
    regex = r'('+ r'|'.join([r1, r2, r3, r4, r5])  + r')'
    back = []
    for text in s:
        matches = re.findall(regex, text)
        if len(matches)>0:
            back.append(text)
    if back != [] and back is not None:
        back = ', '.join(back)
        return back.replace(',', '').replace('(', '').replace(')', '')
    return ''

In [56]:
facebooks = [facebook_url(s) for s in group_info]

In [58]:
#email
def emails(s):
    s = s.lower().split()
    regex = r'[^)].+@.+'
    back = []
    for text in s:
        matches = re.findall(regex, text)
        if len(matches)>0:
            back.append(text)
    if back != [] and back is not None:
        return ', '.join(back)
    return ''


In [59]:
emails = [emails(s) for s in group_info]

In [61]:
def twitter(s):
    #flag = 'twitter' in s.lower()
    s = s.lower().split()
    r1 = r'^@.*[^\s]'
    r2 = r'https://twitter.com/.*[^\s]'
    regex = r'('+ r'|'.join([r1, r2])  + r')'
    back = []
    for text in s:
        matches = re.findall(regex, text)
        if len(matches)>0:
            back.append(text)
    back = ', '.join(back)
    if back != '' and back is not None:
        return back.replace(',','')
    return ''

In [62]:
twitters = [twitter(t) for t in group_info]

In [64]:
def test_web(site):
    try:
        r = urllib.urlopen(site).read()
        return True
        #response is now a string you can search through containing the page's html
    except:
        try:
            r = urllib.urlopen('http://' + site).read()
            return True
        except:
            try:
                r = urllib.urlopen('http://www.' + site).read()
                return True
            except:
                return False
                #The url wasn't valid

In [65]:
def website(s, verbose=True):
    s = s.lower().split()
    r1 = r'.*@.*'
    r2 = r'.*facebook.*'
    r3= r'.*twitter.*'
    expr = [r1, r2, r3]
    regex = re.compile(r'|'.join(expr))
    back = []
    for text in s:
        text = text.replace(',','')
        if len(text) > 5 and ('.' in text) and \
        not bool(regex.match(text)) and \
        test_web(text) and \
        ('gmail' not in text) and \
        text[-1] != '.':
            if verbose:
                print text
            back.append(text)
    if back != [] and back is not None:
        return ''.join(back)
    return ''

In [66]:
websites = [website(w) for w in group_info]

https://www.communitiesrisingus.org/
https://www.meetup.com/communities-rising/
tcprogressives.org
www.perintondemocrats.com
indivisibleaustin.com
https://groups.google.com/forum/#!forum/poinsettgroup
http://savementalhealthreform.org
fortyninemoons.org
http://wearerocklandunited.com
indivisiblemontgomery.wordpress.com
http://www.microaction.us
https://groups.google.com/forum/m/#!forum/indivisible-western-ny
www.writersresist.com
www.northberkeleyforum.org
oregonnastywoman.org
leelanaudemocrats.org
rebuildthehope.org
factstrumpfears.com
progressivenationwnc.org
https://goo.gl/forms/0danjzcj3szy4ncr2
backbonecampaign.org
https://actionnetwork.org/groups/cd-16-citizens-committee
http://www.nwgsdpdx.com/
www.musiciansindivisible.org
ofa.com
100tpc.org
http://pantsuitbellingham.org
www.missoularises.com
http://www.weveseenthisbefore.org/49953/unleash_your_activist_in_downtown_silver_spring_md?recruiter_id=49953
www.tpmobilization.org
asinglevoice.org
http://www.kutztowndemocrats.org
www.de

In [1]:
import hashlib
import random
assert hashlib.sha1('esbinkley@gmail.com').hexdigest() == hashlib.sha1('esbinkley@gmail.com').hexdigest()

In [75]:
key = [hashlib.sha1(w + str(random.randint(0, 1000))).hexdigest() for w in data['Timestamp (EST)']]

In [78]:
def phone_numbers(s):
    s = s.lower()
    regex = r'\d{3}[\.\s-]\d{3}[\.\s-]\d{4}'
    back = []
    matches = re.findall(regex, s)
    if len(matches)==1:
        return matches[0]
    return ''

In [79]:
phones = [phone_numbers(s) for s in group_info]

In [81]:
#with phones
columns=['email', 'facebook', 'twitter', 'website', 'phone']
combine = pd.DataFrame([emails, facebooks, twitters, websites, phones]).transpose()
key = pd.DataFrame(key)
key.columns = ['key']
combine.columns = columns
data_plus_new = pd.concat([key, data, combine], axis=1)
data_plus_new.to_excel('/Users/michaelbinkley/Desktop/old_registered_groups_with_phone.xlsx')

In [82]:
#without phones
columns=['email', 'facebook', 'twitter', 'website']
combine = pd.DataFrame([emails, facebooks, twitters, websites]).transpose()
key = pd.DataFrame(key)
key.columns = ['key']
combine.columns = columns
data_plus_new = pd.concat([key, data, combine], axis=1)
data_plus_new.to_excel('/Users/michaelbinkley/Desktop/old_registered_groups.xlsx')